# SLME - Workflows

To calculate the SLME, we require the absorption coefficient of the material, as well as the direct and indirect band gap. As the CuAu and chalcopyrite materials we investigate all have direct band gaps in the $\Gamma$ point, we can obtain a very accurate band gap from the optics calculation itself.

In [1]:
# Notebook header

import os
from pymatgen import Structure
from custodian.vasp.handlers import VaspErrorHandler
from vscworkflows.config import load_config
from vscworkflows.handlers.core import ElectronicConvergenceMonitor
from vscworkflows.fireworks.core import OptimizeFW, OpticsFW
from vscworkflows.workflows.core import _set_up_functional_params, \
    _set_up_relative_directory, Workflow

First we will define the directory in which we will perform the calculations on the cluster. Note that if you want to run this notebook to replicate my results, you will have to adjust this directory!

In [2]:
cluster = "hopper"

# Set up the scratch dir
if cluster == "breniac":
    scratch_dir = "/scratch/leuven/202/vsc20248/slme/high-throughput"   # This should be changed to your directory
else:
    scratch_dir = "/scratch/antwerpen/202/vsc20248/slme/high-throughput"   # This should be changed to your directory

Next we'll load the launchpad on which we will store the fireworks. If you haven't configured a launchpad yet, open a terminal and use `vsc config launchpad` to do so. The command below loads the 'base' launchpad (i.e. the default). Change this if necessary.

In [3]:
# Load the base launchpad
lpad = load_config("launchpad", cluster)                # Load whichever launchpad you have configured 

After running the [`structures.ipynb`](structures.ipynb) notebook, you should have all structure files stored in the `structures` directory. The line of code below makes a list of all the `.json` files in this directory.

In [4]:
structure_files = [file for file in os.listdir("structures") if ".json" in file]

To submit the workflows to the mongoDB server specified by the `Launchpad` defined above, we will loop over all of the structure files and generate a workflow using the `get_wf_optics` method. This is done in the cell below.

In [5]:
# Functional
# functional=("pbe", {})
functional=("hse06", {})
number_nodes = 8

scratch_dir = "/scratch/antwerpen/202/vsc20248/slme/test"

# Set the error handlers
handlers = [
    VaspErrorHandler(errors_subset_to_catch=["zbrent"]),
    ElectronicConvergenceMonitor()
]

# Change default settings, for e.g. parallelization
user_incar_settings = {
    "KPAR": 2,
    "NPAR": 5,
    "ALGO": "Normal"
}

user_kpoints_settings_optimize = {"reciprocal_density": 100}
user_kpoints_settings_optics = {"gamma_density": 500}

wf_list = []

for file in structure_files:
    
    structure = Structure.from_file(os.path.join("structures", file)) 
    directory = os.path.join(scratch_dir, 
                             file.split(".")[0].split("_")[1],
                             file.split(".")[0].split("_")[0])
    
    # Add number of nodes to spec, or "none"
    if number_nodes is not None and number_nodes != 0:
        spec = {"_fworker": str(number_nodes) + "nodes"}
    else:
        spec = {}
    
    # 1. Set up the geometry optimization with PBE
    vasp_input_params = _set_up_functional_params(
        functional=("pbe", {})
    )
    spec.update(
        {"_launch_dir": _set_up_relative_directory(directory, ("pbe", {}), "optimize")}
    )

    # Override the settings with the user specifications
    if user_kpoints_settings_optimize is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optimize
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the Firework
    optimize_fw_pbe = OptimizeFW(structure=structure,
                             vasp_input_params=vasp_input_params,
                             custodian=handlers,
                             spec=spec)

    # 2. Set up the geometry optimization with chosen functional
    vasp_input_params = _set_up_functional_params(functional)
    spec.update(
        {"_launch_dir": _set_up_relative_directory(directory, functional, "optimize")}
    )

    # Override the settings with the user specifications
    if user_kpoints_settings_optimize is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optimize
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the Firework
    optimize_fw = OptimizeFW(parents=optimize_fw_pbe,
                             vasp_input_params=vasp_input_params,
                             custodian=handlers,
                             spec=spec)

    # 3. Set up the optics calculation
    vasp_input_params = _set_up_functional_params(functional)
    spec.update({"_launch_dir": _set_up_relative_directory(directory, functional,
                                                           "optics")})
    
    # Override the settings with the user specifications
    if user_kpoints_settings_optics is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optics
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the geometry optimization Firework
    optics_fw = OpticsFW(
        parents=optimize_fw,
        vasp_input_params=vasp_input_params,
        custodian=handlers,
        spec=spec
    )

    # Set up a clear name for the workflow
    workflow_name = "Optics "
    workflow_name += str(structure.composition.reduced_formula).replace(" ", "")
    workflow_name += " " + str(functional[0])

    # Create the workflow
    wf_list.append(
        Workflow(fireworks=[optimize_fw_pbe, optimize_fw, optics_fw],
                 links_dict={optimize_fw_pbe: [optimize_fw], optimize_fw: [optics_fw]},
                 name=workflow_name)
    )

/Users/mbercx/miniconda3/envs/vsc/lib/python3.8/site-packages/pymatgen/io/vasp/sets.py:406: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(


In [7]:
for i, l in enumerate(wf_list):
    print(i)
    print(l.as_dict()["fws"][0]["spec"]["_launch_dir"])

0
/scratch/antwerpen/202/vsc20248/slme/test/122/CuAlS2/pbe_optimize
1
/scratch/antwerpen/202/vsc20248/slme/test/115/CuGaSe2/pbe_optimize
2
/scratch/antwerpen/202/vsc20248/slme/test/122/AgGaTe2/pbe_optimize
3
/scratch/antwerpen/202/vsc20248/slme/test/115/CuAlS2/pbe_optimize
4
/scratch/antwerpen/202/vsc20248/slme/test/122/CuGaSe2/pbe_optimize
5
/scratch/antwerpen/202/vsc20248/slme/test/115/AgGaTe2/pbe_optimize
6
/scratch/antwerpen/202/vsc20248/slme/test/115/AgGaSe2/pbe_optimize
7
/scratch/antwerpen/202/vsc20248/slme/test/122/CuGaTe2/pbe_optimize
8
/scratch/antwerpen/202/vsc20248/slme/test/122/CuGaS2/pbe_optimize
9
/scratch/antwerpen/202/vsc20248/slme/test/122/AgInS2/pbe_optimize
10
/scratch/antwerpen/202/vsc20248/slme/test/122/AgGaSe2/pbe_optimize
11
/scratch/antwerpen/202/vsc20248/slme/test/115/CuGaTe2/pbe_optimize
12
/scratch/antwerpen/202/vsc20248/slme/test/115/CuGaS2/pbe_optimize
13
/scratch/antwerpen/202/vsc20248/slme/test/115/AgInS2/pbe_optimize
14
/scratch/antwerpen/202/vsc20248/s

In [9]:
# lpad.bulk_add_wfs(wf_list)
lpad.add_wf(wf_list[34])

2020-01-25 08:36:09,884 INFO Added a workflow. id_map: {-105: 4, -104: 5, -103: 6}


{-105: 4, -104: 5, -103: 6}